# New naver crawling code

In [16]:

import openpyxl
import os
import sys
import time
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains


def naver_map_crawling(address, name):
    url = 'https://map.naver.com/v5/search'
    driver = webdriver.Chrome()  # 드라이버 경로
    # driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
    driver.get(url)
    
    time.sleep(12)
    
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(address)  # 검색어 입력
    search.send_keys(Keys.ENTER)
    
    time.sleep(3)
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(" "+name)  # 검색어 입력
    search.send_keys(Keys.ENTER)
    time.sleep(5)

    #바로 나오는 경우  ex 서울 동대문구 경희대로6길 3-2 에잇올리
    try:
        driver.switch_to.frame("entryIframe")
 
    
    except:

        #바로 안나오는경우 ex) 여기로
        driver.switch_to.frame("searchIframe")
        #첫번째 목록 클릭
        time.sleep(2)
        find_store_click = driver.find_element(By.CSS_SELECTOR,"#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1)")
        find_store_click.click()
    

        time.sleep(2)
        driver.switch_to.default_content()
        driver.switch_to.frame("entryIframe")
        
    #메뉴 클릭
    find_menu_click = driver.find_element(By.XPATH, "//span[text()='메뉴']/parent::a")
    find_menu_click.click()

    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    menu_list = []

    menus = soup.find_all('div',class_ = 'tit')
    for menu in menus:
        menu_list.append(menu.text)
    
    menus = soup.find_all('span',class_ = 'Sqg65')
    for menu in menus:
        menu_list.append(menu.text)
    
    return menu_list
    

In [17]:
def crawling_menus(df):
    menu_list = naver_map_crawling(df['지역'],df['식당이름'])
    
    tmp_row = len(menu_list)
    adr_list = [df['지역'] + " " + df['식당이름']]*tmp_row
    
    
    result = pd.DataFrame({"매장": adr_list, "메뉴":menu_list})
    
    return result

def zacard(A,B):
    intersection_cardinality = len(set.intersection(*[set(A), set(B)]))
    union_cardinality = len(set.union(*[set(A), set(B)]))
    similar = intersection_cardinality / float(union_cardinality)
    
    return similar

def find_ingred(menu_name,recipe):
    eps = 0.6
    main_score=0
    main_menu=""
    
    for menu in recipe['음식']:
        tmp_score = zacard(menu, menu_name)
        
        if tmp_score > main_score:
            main_score = tmp_score
            main_menu = menu
            
    if main_score < eps:
        main_menu="not exsit"
    return main_menu

def matching_menu_ingred(crawled_menu, recipe):
    #레시피 불러오기
    #adr=""
    #recipe = pd.read_csv(adr+'통합레시피_유니크.csv',encoding = 'euc-kr')
    ingred_list =[]
    matched_word_list=[]

    for i in range(len(crawled_menu)):
    
        tmp1 = find_ingred(crawled_menu['메뉴'][i],recipe)
        
        if tmp1 == "not exsit":
            #다시한번 돌고 없으면 not existed
            ######
            tmp2 = check_reverse(crawled_menu['메뉴'][i],recipe)
            if tmp2=="not exist":
                ingred_list.append(tmp1)
                matched_word_list.append(tmp1)
            else:
                ingred_list.append(recipe[recipe['음식']==tmp2]['재료'].item())
                matched_word_list.append(tmp2)

        else:
            ingred_list.append(recipe[recipe['음식']==tmp1]['재료'].item())
            
            matched_word_list.append(tmp1)

    ingred_list=pd.DataFrame({'재료':ingred_list})
    crawled_menu['재료'] = ingred_list
    crawled_menu['매칭된 단어'] = matched_word_list
    return crawled_menu

def crawling_and_matching(df, recipe):
    crawled = crawling_menus(df)
    
    result =matching_menu_ingred(crawled, recipe)
    
    return result

In [13]:
def check_reverse(menu,recipe):
    tmp=0

    for i in recipe['음식']:
        if i in menu:
            tmp=1
        
            return i

    else:
        return "not exist"

In [45]:
df= pd.DataFrame({'지역': ["서울 동작구 성대로 24"],'식당이름':['여기로']})

recipe = pd.read_csv('통합레시피_유니크2.csv',encoding='euc-kr')
crawling_and_matching(df, recipe)

,매장,메뉴,재료,매칭된 단어
0,0 서울 동작구 성대로 24 여기로 dtype: object,퀘사디아,"{'콘', '고추장', '닭고기', '황태', '마늘다진마늘', '새우', '과일'...",퀘사디아
1,0 서울 동작구 성대로 24 여기로 dtype: object,고르곤졸라피자,"{'시판용', '간마늘', '시금치또띠아', '홍고추', '바질', '배', '후추...",고르곤졸라피자
2,0 서울 동작구 성대로 24 여기로 dtype: object,치즈모듬,"{'청양고춧가루', '양파', '아스파라거스', '올리브오일', '참크래커', '슈...",치즈
3,0 서울 동작구 성대로 24 여기로 dtype: object,화로오돌뼈+날치알주먹밥,"{'참기름', '맛술', '김밥용김', '양파', '잘게', '통깨', '김가루',...",날치알주먹밥
4,0 서울 동작구 성대로 24 여기로 dtype: object,새우감바스,"{'파인애플통조림', '자색', '허브가루', '후추가루', '후추', '편마늘',...",새우감바스
5,0 서울 동작구 성대로 24 여기로 dtype: object,챱스테이크,"{'물컵', '브로콜리', '생강', '어린잎채소', '퓨어', '육우', '백미'...",스테이크
6,0 서울 동작구 성대로 24 여기로 dtype: object,계절과일들의합창,"{'딸기', '청포도적포도', '오렌지', '여러', '사과', '플레인', '망고...",과일
7,0 서울 동작구 성대로 24 여기로 dtype: object,제육두부김치,"{'매실원액', '고춧가루', '양파', '후춧가루', '고추장', '김치', '올...",제육두부김치
8,0 서울 동작구 성대로 24 여기로 dtype: object,치즈스틱모듬,"{'스트링', '뜨거운', '매운맛', '체다슬라이스치즈', '후추', '춘권피',...",치즈스틱
9,0 서울 동작구 성대로 24 여기로 dtype: object,국물떡볶이 +길거리튀김 셑,"{'춘장짜장가루', '굵은고추가루', '어묵탕', '죠스떡볶이', '브로콜리', '...",떡볶이


In [41]:
recipe[recipe['음식']=="치즈"]

,음식,재료
3516,치즈,"{'청양고춧가루', '양파', '아스파라거스', '올리브오일', '참크래커', '슈..."


In [23]:
# + 처리
# 칼국수, 밀국수, 쌀국수 구분?
# 돈삼겹 우삼겹 이런건 어땋게 구분할지?

In [ ]:
matching 된거랑 차집합 해서 그거 food_dict안에 있으면 재료로 추가

# FO 메뉴수집

In [21]:
df= pd.DataFrame({'지역': ['서울특별시 마포구 동교로46길'],'식당이름':['이관우파스타']})

In [22]:
crawled = crawling_menus(df)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq.icT4K > a:nth-child(1)"}
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x003ADCE3]
	(No symbol) [0x003439D1]
	(No symbol) [0x00254DA8]
	(No symbol) [0x0028019F]
	(No symbol) [0x002803AB]
	(No symbol) [0x002AEE62]
	(No symbol) [0x0029AF14]
	(No symbol) [0x002AD57C]
	(No symbol) [0x0029ACC6]
	(No symbol) [0x00276F68]
	(No symbol) [0x002780CD]
	GetHandleVerifier [0x00623832+2506274]
	GetHandleVerifier [0x00659794+2727300]
	GetHandleVerifier [0x0065E36C+2746716]
	GetHandleVerifier [0x00456690+617600]
	(No symbol) [0x0034C712]
	(No symbol) [0x00351FF8]
	(No symbol) [0x003520DB]
	(No symbol) [0x0035C63B]
	BaseThreadInitThunk [0x76B97D69+25]
	RtlInitializeExceptionChain [0x77C5BB9B+107]
	RtlClearBits [0x77C5BB1F+191]


In [20]:
crawled

,매장,메뉴
0,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,까누보나라
1,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,봉골레
2,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,알리오올리오
3,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,해물크림리조또
4,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,버섯샐러드
5,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,제노베제
6,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,쵸비알리오
7,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,오징어알리오
8,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,뽀모도로
9,0 서울특별시 마포구 동교로46길 이관우파스타 dtype: object,아라비아따


In [37]:
df_list= pd.read_excel("C:/Users/KEARNEY/Desktop/새 폴더/c_project/LDH/data/양식_주소_이름.xlsx")

In [38]:
df_list.head()

,매장,주소
0,미장플라쎄,서울특별시 마포구 양화로19길
1,이관우파스타,서울특별시 마포구 동교로46길
2,디트로이트 피지스,서울특별시 마포구 동교로46길
3,피크닉 마나올라나,서울특별시 마포구 성미산로
4,키노키친,서울특별시 마포구 와우산로29길


In [39]:
crawled_menu = pd.DataFrame({'지역': [0],'식당이름':[0]})
for i in range(len(df_list)):
    addr = df_list['주소'][i]
    store = df_list['매장'][i]
    
    df= pd.DataFrame({'지역': [addr],'식당이름':[store]})
    crawled = crawling_menus(df)
    crawled_menu = pd.concat([crawled_menu, crawled], axis=0)

In [42]:
result = crawled_menu.iloc[:,2:]

In [43]:
result.to_excel('FO_FW_메뉴정리.xlsx',index=False)